# MultiL-Level Speed-UP

### Classes and modules

In [1]:
#Lets have matplotlib "inline"
%matplotlib inline

import os
import sys

#Import packages we need
import numpy as np
import datetime
from IPython.display import display
import copy

#For plotting
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

plt.style.use('dark_background')

plt.rcParams["image.origin"] = "lower"

import pycuda.driver as cuda

GPU Ocean-modules:

In [2]:
from gpuocean.utils import IPythonMagic, Common

In [3]:
%cuda_context_handler gpu_ctx

In [4]:
ls = [6, 7, 8, 9]

In [5]:
from gpuocean.utils import DoubleJetCase

args_list = []
init_list = []

for l in ls:
    doubleJetCase = DoubleJetCase.DoubleJetCase(gpu_ctx, DoubleJetCase.DoubleJetPerturbationType.SteadyState, ny=2**l, nx=2**(l+1))
    doubleJetCase_args, doubleJetCase_init, _ = doubleJetCase.getInitConditions()

    args_list.append(doubleJetCase_args)
    init_list.append(doubleJetCase_init)

### Set Work (pracical or theoretical)

In [6]:
work_path = "/home/florianb/havvarsel/multilevelDA/doublejet/scripts/PracticalCost/2023-09-18T12_54_47"

works = np.mean(np.load(work_path+"/costsPure.npy"), axis=-1)
diff_works = np.mean(np.load(work_path+"/costsPartnered.npy"), axis=-1)[1:]

Comment next cell to use practical work

In [7]:
works = [1/(8**3), 1/(8**2), 1/(8), 1]
diff_works = [1/(8**2), 1/(8), 1]

In [8]:
works, diff_works

(array([0.33697817, 0.48686108, 1.1457611 , 7.20046325]),
 array([0.86028268, 1.4491079 , 8.37040459]))

### Set variances (practical)

In [9]:
source_path = "/home/florianb/havvarsel/multilevelDA/doublejet/scripts/VarianceLevelsDA/2023-10-27T17_43_20mean"
# source_path = "/home/florianb/havvarsel/multilevelDA/doublejet/scripts/VarianceLevelsDA/2023-10-27T17_43_20var"

In [10]:
vars = np.load(source_path+"/vars_"+str(10*24*3600)+".npy")
diff_vars = np.load(source_path+"/diff_vars_"+str(10*24*3600)+".npy")

## Ensembles with equal error $\tau$

In [11]:
# var_idx = 0 ->  eta
# var_idx = 1 ->  hu
# var_idx = 2 ->  hv

In [12]:
def speedUp(var_idx, SL_Ne = 50, N_l=None):
    if N_l is None:
        N_l = len(vars)
    # vars only for one variable
    v_vars = vars[:,var_idx][-N_l:]
    v_diff_vars = diff_vars[:,var_idx][-N_l+1:]

    v_works = works[-N_l:]
    v_diff_works = diff_works[-N_l+1:]

    # tau corresponding to SL with SL_Ne
    tau = np.sqrt(v_vars[-1]/SL_Ne)

    # ML ensemble size
    allwork = np.sqrt(v_vars[0] * v_works[0])
    for k_idx in range(1, N_l):
        allwork += np.sqrt(v_diff_vars[k_idx-1] * v_diff_works[k_idx-1])

    ML_Ne = np.zeros(N_l)
    ML_Ne[0] = 1/(tau**2)*np.sqrt(v_vars[0]/v_works[0]) * allwork
    for l_idx in range(1, N_l):
        ML_Ne[l_idx] = 1/(tau**2)*np.sqrt(v_diff_vars[l_idx-1]/v_diff_works[l_idx-1]) * allwork

    ML_Ne = np.ceil(ML_Ne).astype(int)

    # SL work 
    # = SL_Ne * work(SL)
    SLwork = SL_Ne * v_works[-1]

    # ML work
    MLwork = ML_Ne[0]*v_works[0]
    for l_idx in range(1, len(ML_Ne)):
        MLwork += ML_Ne[l_idx]*v_diff_works[l_idx-1]

    # speed up
    speed_up = SLwork/MLwork
    
    return ML_Ne, speed_up, tau

In [13]:
for i in range(3):
    for N_l in range(2, len(ls)+1):
        print(speedUp(i, N_l=N_l))

(array([104,  38]), 0.8234097236886546, 9.553522644651816)
(array([106,  82,  41]), 0.7009514124608921, 9.553522644651816)
(array([54, 73, 86, 43]), 0.6365915488775665, 9.553522644651816)
(array([87, 20]), 1.3479504887515406, 419903.75789049565)
(array([109,  64,  22]), 1.0911126465400132, 419903.75789049565)
(array([32, 81, 68, 23]), 0.9690419651366244, 419903.75789049565)
(array([79, 20]), 1.3958540235138657, 367099.08705220174)
(array([62, 57, 21]), 1.2476413137945472, 367099.08705220174)
(array([26, 44, 60, 22]), 1.133184405430159, 367099.08705220174)
